In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

c:\Users\jaesc2\GitHub\skforecast


In [1]:
import time
import platform
import psutil
import sklearn
import numpy as np
import pandas as pd
import lightgbm
import skforecast
from packaging.version import parse as parse_version

from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler
from skforecast.utils import check_predict_input
from skforecast.recursive import ForecasterRecursive
from skforecast.model_selection import TimeSeriesFold, backtesting_forecaster, backtesting_forecaster_multiseries

%load_ext pyinstrument
%load_ext line_profiler

In [2]:
print(f"Python version: {platform.python_version()}")
print(f"skforecast version: {skforecast.__version__}")
print(f"scikit-learn version: {sklearn.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"Computer network name: {platform.node()}")
print(f"Processor type: {platform.processor()}")
print(f"Platform type: {platform.platform()}")
print(f"Operating system: {platform.system()}")
print(f"Operating system release: {platform.release()}")
print(f"Operating system version: {platform.version()}")
print(f"Number of physical cores: {psutil.cpu_count(logical=False)}")
print(f"Number of logical cores: {psutil.cpu_count(logical=True)}")

Python version: 3.12.11
skforecast version: 0.16.0
scikit-learn version: 1.7.1
pandas version: 2.3.1
numpy version: 2.1.3
Computer network name: ITES015-NB0029
Processor type: Intel64 Family 6 Model 141 Stepping 1, GenuineIntel
Platform type: Windows-11-10.0.26100-SP0
Operating system: Windows
Operating system release: 11
Operating system version: 10.0.26100
Number of physical cores: 8
Number of logical cores: 16


In [3]:
# Mock data for benchmarking
# ==========================================================
len_series = 2000
rng = np.random.default_rng(321)
y = pd.Series(
        data = rng.normal(loc=20, scale=5, size=len_series),
        index=pd.date_range(
            start='2010-01-01',
            periods=len_series,
            freq='h'
        ),
        name='y'
    )
rng = np.random.default_rng(321)
exog = pd.DataFrame(index=y.index)
exog['day_of_week'] = exog.index.dayofweek
exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
exog['month'] = exog.index.month
exog_prediction = pd.DataFrame(
                    index=pd.date_range(
                        start=exog.index.max() + pd.Timedelta(hours=1),
                        periods=100,
                        freq='h'
                    )
                 ) 
exog_prediction['day_of_week'] = exog_prediction.index.dayofweek
exog_prediction['week_of_year'] = exog_prediction.index.isocalendar().week.astype(int)
exog_prediction['month'] = exog_prediction.index.month

In [4]:
regressor = LGBMRegressor(random_state=8520, verbose=-1)
forecaster = ForecasterRecursive(
    regressor=regressor,
    lags=50,
    transformer_y=None,
    transformer_exog=None,
)
forecaster.fit(y=y, exog=exog)

## skforecast 0.17.0

In [6]:
%%timeit -n 1000 -r 4

check_predict_input(
    forecaster_name  = type(forecaster).__name__,
    steps            = 100,
    is_fitted        = forecaster.is_fitted,
    exog_in_         = forecaster.exog_in_,
    index_type_      = forecaster.index_type_,
    index_freq_      = forecaster.index_freq_,
    window_size      = forecaster.window_size,
    last_window      = forecaster.last_window_,
    exog             = exog_prediction,
    exog_names_in_   = forecaster.exog_names_in_,
    interval         = None
)

167 μs ± 25.6 μs per loop (mean ± std. dev. of 4 runs, 1,000 loops each)


In [7]:
# skforecast 0.17.0
# ==============================================================================
def funt_to_profile(forecaster, steps, exog):
    check_predict_input(
    forecaster_name  = type(forecaster).__name__,
    steps            = steps,
    is_fitted        = forecaster.is_fitted,
    exog_in_         = forecaster.exog_in_,
    index_type_      = forecaster.index_type_,
    index_freq_      = forecaster.index_freq_,
    window_size      = forecaster.window_size,
    last_window      = forecaster.last_window_,
    exog             = exog,
    exog_names_in_   = forecaster.exog_names_in_,
    interval         = None
)


%lprun -f check_predict_input funt_to_profile(forecaster, 100, exog_prediction)

Timer unit: 1e-07 s

Total time: 0.0029372 s
File: c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py
Function: check_predict_input at line 776

Line #      Hits         Time  Per Hit   % Time  Line Contents
   776                                           def check_predict_input(
   777                                               forecaster_name: str,
   778                                               steps: int | list[int],
   779                                               is_fitted: bool,
   780                                               exog_in_: bool,
   781                                               index_type_: type,
   782                                               index_freq_: str,
   783                                               window_size: int,
   784                                               last_window: pd.Series | pd.DataFrame | None,
   785                                               last_window_exog: pd.Series | pd.DataFrame | None = 

In [8]:
%%timeit -n 1000 -r 4

forecaster._create_predict_inputs(
    steps=100, exog=exog_prediction
)

1.2 ms ± 142 μs per loop (mean ± std. dev. of 4 runs, 1,000 loops each)


In [9]:
# skforecast 0.17.0
# ==============================================================================
def funt_to_profile(steps, exog):
    forecaster._create_predict_inputs(
        steps=steps, exog=exog
    )


%lprun -f forecaster._create_predict_inputs funt_to_profile(100, exog_prediction)

Timer unit: 1e-07 s

Total time: 0.0059891 s
File: c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive.py
Function: ForecasterRecursive._create_predict_inputs at line 1113

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1113                                               def _create_predict_inputs(
  1114                                                   self,
  1115                                                   steps: int | str | pd.Timestamp, 
  1116                                                   last_window: pd.Series | pd.DataFrame | None = None,
  1117                                                   exog: pd.Series | pd.DataFrame | None = None,
  1118                                                   predict_probabilistic: bool = False,
  1119                                                   use_in_sample_residuals: bool = True,
  1120                                                   use_binned_residuals: bool = True,
  1121    

In [10]:
%%timeit -n 10 -r 10

forecaster.predict(
    steps=100, exog=exog_prediction
)

162 ms ± 18.2 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [11]:
# skforecast 0.17.0
# ==============================================================================
def funt_to_profile(steps, exog):
    forecaster.predict(
        steps=steps, exog=exog
    )


%lprun -f forecaster.predict funt_to_profile(100, exog_prediction)

Timer unit: 1e-07 s

Total time: 0.838516 s
File: c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive.py
Function: ForecasterRecursive.predict at line 1449

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1449                                               def predict(
  1450                                                   self,
  1451                                                   steps: int | str | pd.Timestamp,
  1452                                                   last_window: pd.Series | pd.DataFrame | None = None,
  1453                                                   exog: pd.Series | pd.DataFrame | None = None,
  1454                                                   check_inputs: bool = True
  1455                                               ) -> pd.Series:
  1456                                                   """
  1457                                                   Predict n steps ahead. It is an recursive process in 

## skforecast 0.16.0

In [5]:
%%timeit -n 1000 -r 4

check_predict_input(
    forecaster_name  = type(forecaster).__name__,
    steps            = 100,
    is_fitted        = forecaster.is_fitted,
    exog_in_         = forecaster.exog_in_,
    index_type_      = forecaster.index_type_,
    index_freq_      = forecaster.index_freq_,
    window_size      = forecaster.window_size,
    last_window      = forecaster.last_window_,
    exog             = exog_prediction,
    exog_names_in_   = forecaster.exog_names_in_,
    interval         = None
)

143 μs ± 9.1 μs per loop (mean ± std. dev. of 4 runs, 1,000 loops each)


In [6]:
# skforecast 0.16.0
# ==============================================================================
def funt_to_profile(forecaster, steps, exog):
    check_predict_input(
        forecaster_name  = type(forecaster).__name__,
        steps            = steps,
        is_fitted        = forecaster.is_fitted,
        exog_in_         = forecaster.exog_in_,
        index_type_      = forecaster.index_type_,
        index_freq_      = forecaster.index_freq_,
        window_size      = forecaster.window_size,
        last_window      = forecaster.last_window_,
        exog             = exog,
        exog_names_in_   = forecaster.exog_names_in_,
        interval         = None
    )


%lprun -f check_predict_input funt_to_profile(forecaster, 100, exog_prediction)

Timer unit: 1e-07 s

Total time: 0.0039462 s
File: c:\Users\jaesc2\Miniconda3\envs\skforecast_py12\Lib\site-packages\skforecast\utils\utils.py
Function: check_predict_input at line 773

Line #      Hits         Time  Per Hit   % Time  Line Contents
   773                                           def check_predict_input(
   774                                               forecaster_name: str,
   775                                               steps: int | list[int],
   776                                               is_fitted: bool,
   777                                               exog_in_: bool,
   778                                               index_type_: type,
   779                                               index_freq_: str,
   780                                               window_size: int,
   781                                               last_window: pd.Series | pd.DataFrame | None,
   782                                               last_window_exog: pd

In [7]:
%%timeit -n 1000 -r 4

forecaster._create_predict_inputs(
    steps=100, exog=exog_prediction
)

1.31 ms ± 57.1 μs per loop (mean ± std. dev. of 4 runs, 1,000 loops each)


In [8]:
# skforecast 0.16.0
# ==============================================================================
def funt_to_profile(steps, exog):
    forecaster._create_predict_inputs(
        steps=steps, exog=exog
    )


%lprun -f forecaster._create_predict_inputs funt_to_profile(100, exog_prediction)

Timer unit: 1e-07 s

Total time: 0.0063473 s
File: c:\Users\jaesc2\Miniconda3\envs\skforecast_py12\Lib\site-packages\skforecast\recursive\_forecaster_recursive.py
Function: ForecasterRecursive._create_predict_inputs at line 1098

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1098                                               def _create_predict_inputs(
  1099                                                   self,
  1100                                                   steps: int | str | pd.Timestamp, 
  1101                                                   last_window: pd.Series | pd.DataFrame | None = None,
  1102                                                   exog: pd.Series | pd.DataFrame | None = None,
  1103                                                   predict_probabilistic: bool = False,
  1104                                                   use_in_sample_residuals: bool = True,
  1105                                                   use_binned_re

In [9]:
%%timeit -n 10 -r 10

forecaster.predict(
    steps=100, exog=exog_prediction
)

155 ms ± 10.8 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [10]:
# skforecast 0.16.0
# ==============================================================================
def funt_to_profile(steps, exog):
    forecaster.predict(
        steps=steps, exog=exog
    )


%lprun -f forecaster.predict funt_to_profile(100, exog_prediction)

Timer unit: 1e-07 s

Total time: 0.9145 s
File: c:\Users\jaesc2\Miniconda3\envs\skforecast_py12\Lib\site-packages\skforecast\recursive\_forecaster_recursive.py
Function: ForecasterRecursive.predict at line 1429

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1429                                               def predict(
  1430                                                   self,
  1431                                                   steps: int | str | pd.Timestamp,
  1432                                                   last_window: pd.Series | pd.DataFrame | None = None,
  1433                                                   exog: pd.Series | pd.DataFrame | None = None,
  1434                                                   check_inputs: bool = True
  1435                                               ) -> pd.Series:
  1436                                                   """
  1437                                                   Predict n steps ahead. 